In [ ]:
import re
from time import sleep
import itertools
import requests
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
import pyarrow as pa
from zipfile import ZipFile

import dotenv
dotenv.load_dotenv(".envrc", override=True)

In [ ]:

URLS = {
    "hist_data": "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/",
    "recent_data": "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/recent/",
}

# for category, url in URLS.items():
#     page = requests.get(url)
#     soup = BeautifulSoup(page.content, "html.parser")

#     links = soup.find_all("a", href=re.compile(".pdf$|.txt$|.zip$"))

#     for link in links:
#         path = Path("./data/zips") / category
#         path.mkdir(parents=True, exist_ok=True)
#         file_path = path / link["href"]
#         mode = "w+b" if "pdf" or "zip" in link["href"] else "w+"
#         file_url = url + link["href"]
#         if not file_path.is_file():
#             with requests.get(file_url) as r:
#                 with open(str(file_path), mode) as f:
#                     f.write(r.content)
#                     sleep(0.5)


In [ ]:
for category in URLS:
    source_path = Path("./data") / category / "zips"
    target_path = Path("./data") / category / "metadata"
    target_path.mkdir(parents=True, exist_ok=True)

    for file in source_path.glob("*.zip"):
        with ZipFile(file, "r") as zip:
            fname_lst = [fname for fname in zip.namelist() if "Metadaten" in fname] # produkt_klima_tag for data, Metadaten for metadata
            for fname in fname_lst:
                zip.extract(fname, target_path)
    

files_to_rem = target_path.glob("*.html")
for file in files_to_rem:
    file.unlink()

In [ ]:
files_to_rem = Path("./data/hist_data/metadata/").glob("*.html")
for file in files_to_rem:
    file.unlink()

## Load main dataset

In [ ]:
# dtypes= {
#     'STATIONS_ID': "string", 
#     'QN_3': "UInt8", 
#     'QN_4': "UInt8", 
#     ' RSK': "Float32", 
#     'RSKF': "UInt8", 
#     'SHK_TAG': "UInt8",
#     '  NM': "string", 
#     ' TMK': "Float32", 
#     ' UPM': "Float32", 
#     ' TXK': "Float32", 
#     ' TNK': "Float32", 
#     ' TGK': "Float32",
# }


# df_new_lst = [pd.read_table(
#         file, 
#         sep=";",
#         usecols=['STATIONS_ID', 'MESS_DATUM', 'QN_3', 'QN_4', ' RSK',
#         'RSKF', 'SHK_TAG', '  NM', ' TMK', ' UPM',
#         ' TXK', ' TNK', ' TGK'],
#         dtype_backend="pyarrow",
#         dtype=dtypes,
#         na_values=None,
#     ) for file in Path("./data/recent_data/extracts").glob("*.txt")]

# df_hist_lst = [pd.read_table(
#         file, 
#         sep=";",
#         usecols=['STATIONS_ID', 'MESS_DATUM', 'QN_3', 'QN_4', ' RSK',
#         'RSKF', 'SHK_TAG', '  NM', ' TMK', ' UPM',
#         ' TXK', ' TNK', ' TGK'],
#         dtype_backend="pyarrow",
#         dtype=dtypes,
#         na_values=None,
#     ) for file in Path("./data/hist_data/extracts").glob("*.txt")]

## Prepare main dataset

In [ ]:
# df = pd.concat(df_new_lst + df_hist_lst)
# df = df.replace(-999, None).astype(dtypes)
# df["MESS_DATUM"] = pd.to_datetime(df["MESS_DATUM"], format="%Y%m%d", errors="coerce", utc=False).dt.tz_localize("Europe/Brussels", ambiguous = "NaT", nonexistent="NaT")
# df.columns = df.columns.str.replace(' ', '')
# df.info()

In [ ]:
# df.to_parquet("./data/parquet/data.parquet")
# df = pd.read_parquet("./data/parquet/data.parquet", dtype_backend="pyarrow")
# df.info()

## Load and prepare metadata

In [ ]:
# dtypes = {
#     'Stations_id': "string",
#     'Stationshoehe': "Float32",
#     'Geogr.Breite': "Float32",
#     'Geogr.Laenge': "Float32",
#     'von_datum': "string",
#     'bis_datum': "string",
#     'Stationsname': "string",
# }

# df_new_lst = [pd.read_table(
#         file, 
#         sep=";",
#         encoding="latin1",
#         dtype=dtypes,
#         dtype_backend="pyarrow",
#         na_values=None) for file in Path("./data/recent_data/metadata").glob("Metadaten_Geographie*")]

# df_hist_lst = [pd.read_table(
#         file, 
#         sep=";",
#         encoding="latin1",
#         dtype=dtypes,
#         dtype_backend="pyarrow",
#         na_values=None) for file in Path("./data/hist_data/metadata").glob("Metadaten_Geographie*")]

# df = pd.concat(df_new_lst + df_hist_lst)
# df.Stations_id = df.Stations_id.str.replace(' ', '')
# df = df.replace(-999, None).astype(dtypes)
# df["von_datum"] = pd.to_datetime(df["von_datum"], format="%Y%m%d", errors="coerce", utc=False).dt.tz_localize("Europe/Brussels", ambiguous = "NaT")
# df["bis_datum"] = pd.to_datetime(df["bis_datum"], format="%Y%m%d", errors="coerce", utc=False).dt.tz_localize("Europe/Brussels", ambiguous = "NaT")
# df.columns = df.columns.str.replace(' ', '')

In [ ]:
# df.to_parquet("./data/parquet/metadata_geo.parquet")
# df = pd.read_parquet("./data/parquet/metadata_geo.parquet", dtype_backend="pyarrow")
# df.info()

In [ ]:
# df_new_lst = [pd.read_table(
#         file, 
#         sep=";",
#         encoding="latin1",
#         dtype_backend="pyarrow",
#         na_values=None) for file in Path("./data/recent_data/metadata").glob("Metadaten_Stationsname_Betreibername*")]
# df_lst = []
# for df in df_new_lst:
#         split_idx = df.loc[df['Stationsname']=="Betreibername"].index[0]
#         df.columns = ["stations_id", "betreibername", "betrieb_von_datum", "betrieb_bis_datum"]
#         df = df.iloc[split_idx+1:-1]
#         df_lst.append(df)

# df = pd.concat(df_lst).reset_index(drop=True)
# df.stations_id = df.stations_id.str.replace(' ', '')
# df["betrieb_von_datum"] = pd.to_datetime(df["betrieb_von_datum"], format="%Y%m%d", errors="coerce", utc=False).dt.tz_localize("Europe/Brussels", ambiguous = "NaT")
# df["betrieb_bis_datum"] = pd.to_datetime(df["betrieb_bis_datum"], format="%Y%m%d", errors="coerce", utc=False).dt.tz_localize("Europe/Brussels", ambiguous = "NaT")

In [ ]:
# df.to_parquet("./data/parquet/metadata_operator.parquet")
# df = pd.read_parquet("./data/parquet/metadata_operator.parquet", dtype_backend="pyarrow")
# df.info()

## Spark

In [ ]:
# from pyspark.sql import SparkSession
# import pyspark.sql.types as T
# import pyspark.sql.functions as F

# spark.conf.set('spark.sql.legacy.parquet.datetimeRebaseModeInRead', 'CORRECTED')

# schema = T.StructType([
#     T.StructField('STATIONS_ID', T.StringType(), False),
#     T.StructField('MESS_DATUM', T.TimestampType(), True),
#     T.StructField('QN_3', T.ShortType(), True),
#     T.StructField('QN_4', T.ShortType(), True),
#     T.StructField('RSK', T.DoubleType(), True),
#     T.StructField('RSKF', T.ShortType(), True),
#     T.StructField('SHK_TAG', T.IntegerType(), True),
#     T.StructField('NM', T.StringType(), True),
#     T.StructField('TMK', T.DoubleType(), True),
#     T.StructField('UPM', T.DoubleType(), True),
#     T.StructField('TXK', T.DoubleType(), True),
#     T.StructField('TNK', T.DoubleType(), True),
#     T.StructField('TGK', T.DoubleType(), True),
# ])

# spark = SparkSession.builder \
#     .master("local[*]") \
#     .appName('SparkNotebook') \
#     .getOrCreate()

In [ ]:
# data_path = Path("./data/data.parquet")

In [ ]:
# df = spark.read \
#     .option("header", "true") \
#     .parquet(str(data_path))

# df.createOrReplaceTempView("temps")


In [ ]:
# df.printSchema()

In [ ]:
# results = spark.sql("""
# SELECT distinct(extract(year from MESS_DATUM)) from temps
# where date(MESS_DATUM) >= "1900-01-01"
# order by 1 desc
# """)
# results.show(40, False)

In [ ]:
# results = spark.sql("""
# SELECT STATIONS_ID,extract(year from MESS_DATUM), max(TXK) as max_temp_2m from temps
# group by 1,2
# order by 3 desc
# """)
# results.show(40, False)